In [21]:
# -*- coding: utf-8 -*-
"""
Adaptive Skill Assessment Model - Simulation with Updated Dataset
"""

# --- Step 1: Import Libraries and Modules ---
import pandas as pd
import random
import os
import sys

# Robust path setup (for both /scripts and /data folders)
current_dir = os.getcwd()
if 'notebooks' in current_dir.split(os.sep):
    sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'scripts')))
else:
    sys.path.append(os.path.abspath(os.path.join(current_dir, 'scripts')))

# Import project modules
try:
    from adaptive_logic import AdaptiveEngine
    from jobfit_predictor import JobFitPredictor
    print("✅ Project modules loaded successfully.")
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    class AdaptiveEngine: pass
    class JobFitPredictor: pass

# --- Step 2: Load the Dataset ---
csv_path = r"D:\Citi_Hackathon\data\assessment_data.csv"

try:
    df = pd.read_csv(csv_path)
    if df.empty:
        raise ValueError("Dataset appears empty!")

    # Ensure Q_ID exists
    if 'Q_ID' not in df.columns:
        df.insert(0, 'Q_ID', range(1, len(df) + 1))

    # AdaptiveEngine will handle Difficulty column dynamically
    print(f"\n✅ Dataset Loaded Successfully from: {csv_path}")
    print(df.head())

except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    # Fallback minimal dataset
    df = pd.DataFrame({
        'Q_ID': [1, 2, 3, 4],
        'Job_Role': ['AI/ML Engineer', 'DevOps Engineer', 'Cloud Engineer', 'Frontend Developer'],
        'Skill': ['Python', 'CI/CD', 'AWS', 'HTML'],
        'Question': ['What is ML?', 'What is CI?', 'What is AWS?', 'What is HTML?'],
        'Options': ['A;B;C', 'X;Y;Z', '1;2;3', 'D;E;F'],
        'Answer': ['A', 'Y', '1', 'D'],
        'Difficulty_Level': [2, 3, 1, 1]
    })
    print("Using fallback dataset for simulation.")

# --- Step 3: Initialize Components ---
roles = df['Job_Role'].unique().tolist()
print("\n🎯 Roles Detected:", roles)

engine, predictor = None, None
try:
    engine = AdaptiveEngine(df)
    predictor = JobFitPredictor(model_dir='models')
    print("\n✅ Adaptive Engine and Job Fit Predictor Initialized.")
except Exception as e:
    print(f"⚠️ Initialization failed: {e}")

# --- Step 4: Simulation Function ---
def run_adaptive_simulation(role_name, engine_instance, predictor_instance, num_questions=8, fixed_trust_score=85):
    if not isinstance(engine_instance, AdaptiveEngine) or not isinstance(predictor_instance, JobFitPredictor):
        print("Skipping simulation: Engine or Predictor not initialized.")
        return None

    print(f"\n{'='*60}")
    print(f"🎯 STARTING SIMULATION for Role: {role_name} ({num_questions} questions max)")
    print(f"{'='*60}")

    current_score = 0
    q_data = engine_instance.get_initial_question(role_name)

    for i in range(num_questions):
        if q_data is None:
            print("⚠️ No more questions available.")
            break

        q_id = q_data['Q_ID']
        question = q_data['Question']
        options = q_data['Options'].split(';')
        correct_answer = q_data['Answer']
        difficulty = q_data['Difficulty']

        predicted_answer = random.choice(options)
        is_correct = (predicted_answer.strip() == correct_answer.strip())

        next_q, new_score = engine_instance.update_and_get_next(
            q_id=q_id, is_correct=is_correct, current_score=current_score
        )

        print(f"Q{i+1} (D:{difficulty}) {question[:40]}...")
        print(f"  -> Predicted: {predicted_answer} | {'✅ Correct' if is_correct else '❌ Incorrect'} | New Score: {round(new_score, 2)}")

        current_score = new_score
        q_data = next_q

    # --- Compute Results ---
    final_skill_score = engine_instance.get_final_skill_score(current_score)
    fit_result = predictor_instance.predict_fit(final_skill_score, trust_score=fixed_trust_score)

    print(f"\n{'*'*60}")
    print(f"🎓 ROLE: {role_name}")
    print(f"SKILL SCORE: {fit_result['SkillScore']}% | TRUST: {fit_result['TrustScore']}%")
    print(f"JOB FIT SCORE: {fit_result['JobFitScore']}% → CATEGORY: {fit_result['Category']}")
    print(f"{'*'*60}\n")

    return fit_result

# --- Step 5: Run Simulations for All Roles ---
simulation_results = {}
if engine and predictor:
    for role in roles:
        role_engine = AdaptiveEngine(df)
        result = run_adaptive_simulation(role, role_engine, predictor)
        if result:
            simulation_results[role] = result
else:
    print("\n⚠️ Engine/Predictor not initialized. Skipping simulations.")

# --- Step 6: Display Final Results ---
if simulation_results:
    results_df = pd.DataFrame.from_dict(simulation_results, orient='index').reset_index()
    results_df.rename(columns={'index': 'Job_Role'}, inplace=True)
    print("\n\n📊 AGGREGATED SIMULATION RESULTS:")
    print(results_df[['Job_Role', 'SkillScore', 'TrustScore', 'JobFitScore', 'Category']]
          .sort_values(by='JobFitScore', ascending=False)
          .to_markdown(index=False))
else:
    print("\n❌ No simulation results generated.")


✅ Project modules loaded successfully.

✅ Dataset Loaded Successfully from: D:\Citi_Hackathon\data\assessment_data.csv
   Q_ID          Job_Role    Skill  \
0     1  Mobile Developer  Android   
1     2  Mobile Developer  Android   
2     3  Mobile Developer  Android   
3     4  Mobile Developer      iOS   
4     5  Mobile Developer      iOS   

                                            Question  \
0  Which language is primarily used for Android a...   
1   Which layout is best for complex UI positioning?   
2  Which lifecycle method is called when app is a...   
3               Which language is used for iOS apps?   
4  Which component displays multiple items in a s...   

                                             Options            Answer  \
0                               Java;Kotlin;Swift;C#            Kotlin   
1  LinearLayout;RelativeLayout;ConstraintLayout;F...  ConstraintLayout   
2                  onCreate;onStart;onResume;onPause           onStart   
3                  